# Regresión de LASSO y Ridge

Esta función muestra cómo utilizar TensorFlow para resolver la regresión de LASSO o Ridge para$\boldsymbol{y} = \boldsymbol{Ax} + \boldsymbol{b}$

Usaremos los datos de Iris, específicamente: $\boldsymbol{y}$ = Sepal Length, $\boldsymbol{x}$ = Petal Width

In [ ]:
# importar las bibliotecas requeridas
import matplotlib.pyplot as plt
import sys
import numpy as np
import tensorflow as tf
from sklearn import datasets
from tensorflow.python.framework import ops

In [ ]:
# Especificar 'Ridge' o 'LASSO'
regression_type = 'LASSO'

In [ ]:
# borrar el grafo anterior
ops.reset_default_graph()

# Crear un nuevo grafo
sess = tf.Session()

## Cargar los datos de Iris

In [ ]:
# iris.data = [(Sepal Length, Sepal Width, Petal Length, Petal Width)]
iris = datasets.load_iris()
x_vals = np.array([x[3] for x in iris.data])
y_vals = np.array([y[0] for y in iris.data])

## Modelar los parámetros

In [ ]:
# Declarar el tamaño del lote
batch_size = 50

# Inicializar los marcadores de posición
x_data = tf.placeholder(shape=[None, 1], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# hacer reproducibles los resultados
seed = 13
np.random.seed(seed)
tf.set_random_seed(seed)

# Crear las variables para la Regresión lineal
A = tf.Variable(tf.random_normal(shape=[1,1]))
b = tf.Variable(tf.random_normal(shape=[1,1]))

# Declarar las operaciones del modelo
model_output = tf.add(tf.matmul(x_data, A), b)

## Las funciones de pérdida


In [ ]:
# Seleccionar la función de pérdida apropiada basada en el tipo de regresión

if regression_type == 'LASSO':
    # Declarar la función de pérdida de LASSO
    # Lasso Loss = L2_Loss + heavyside_step,
    # Donde heavyside_step ~ 0 si A < constant, de otra manera ~ 99
    lasso_param = tf.constant(0.9)
    heavyside_step = tf.truediv(1., tf.add(1., tf.exp(tf.multiply(-50., tf.subtract(A, lasso_param)))))
    regularization_param = tf.multiply(heavyside_step, 99.)
    loss = tf.add(tf.reduce_mean(tf.square(y_target - model_output)), regularization_param)

elif regression_type == 'Ridge':
    # Declarar la funciónde pérdida de Ridge
    # Ridge loss = L2_loss + L2 norm of slope
    ridge_param = tf.constant(1.)
    ridge_loss = tf.reduce_mean(tf.square(A))
    loss = tf.expand_dims(tf.add(tf.reduce_mean(tf.square(y_target - model_output)), tf.multiply(ridge_param, ridge_loss)), 0)
    
else:
    print('Invalid regression_type parameter value',file=sys.stderr)


## Optimizador

In [ ]:
# Declarar el optimizador
my_opt = tf.train.GradientDescentOptimizer(0.001)
train_step = my_opt.minimize(loss)

## Ejecutar la regresión

In [ ]:
# Inicializar las variables
init = tf.global_variables_initializer()
sess.run(init)

# Ciclo de entrenamiento
loss_vec = []
for i in range(1500):
    rand_index = np.random.choice(len(x_vals), size=batch_size)
    rand_x = np.transpose([x_vals[rand_index]])
    rand_y = np.transpose([y_vals[rand_index]])
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    temp_loss = sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})
    loss_vec.append(temp_loss[0])
    if (i+1)%300==0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)) + ' b = ' + str(sess.run(b)))
        print('Loss = ' + str(temp_loss))
        print('\n')

## Extraer los resultados de la regresión

In [ ]:
# Obtener los coeficientes óptimos
[slope] = sess.run(A)
[y_intercept] = sess.run(b)

# Obtener la mejor línea de ajuste
best_fit = []
for i in x_vals:
  best_fit.append(slope*i+y_intercept)

## Graficar los resultados

In [ ]:
%matplotlib inline
# Graficar el resultado
plt.plot(x_vals, y_vals, 'o', label='Data Points')
plt.plot(x_vals, best_fit, 'r-', label='Best fit line', linewidth=3)
plt.legend(loc='upper left')
plt.title('Sepal Length vs Pedal Width')
plt.xlabel('Pedal Width')
plt.ylabel('Sepal Length')
plt.show()

# Graficar la función de pérdida
plt.plot(loss_vec, 'k-')
plt.title(regression_type + ' Loss per Generation')
plt.xlabel('Generation')
plt.ylabel('Loss')
plt.show()